In [6]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [7]:
!python -V

Python 3.10.12


In [8]:
import pickle
import pandas as pd

In [9]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [10]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [11]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [12]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [13]:
"Q1: %0.3f" % ( y_pred.std() )

'Q1: 6.247'

In [14]:
year = 2023
month = 3
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [15]:
df_result = pd.DataFrame( {'prediction': y_pred, 'ride_id': df['ride_id']} )

In [16]:
output_file = 'result.pyarrow'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [17]:
# Q2
!ls -lh result*

-rw-rw-r-- 1 sir sir 66M jun 17 21:15 result.pyarrow


In [18]:
# Q3
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 1488 bytes to starter.py
